# 目的
validデータに対する性能を確認する。

ref: 
- https://github.com/unslothai/unsloth
- https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing#scrollTo=6bZsfBuZDeCL
- https://huggingface.co/blog/mlabonne/sft-llama3
- https://huggingface.co/docs/trl/sft_trainer#accelerate-fine-tuning-2x-using-unsloth

In [1]:
# pip install "unsloth[cu121-torch230] @ git+https://github.com/unslothai/unsloth.git"
# -> だめ

# pip uninstall torch
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# pip install "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"
# -> OK

In [2]:
import torch

torch.__version__

'2.4.1+cu121'

In [3]:
# path setting
EXP_NAME = "e001-baseline"
MODEL_NAME = "unsloth/gemma-2-9b-it-bnb-4bit"
COMPETITION_NAME = "eedi-mining-misconceptions-in-mathematics"

DATA_PATH = "data"
DATASET_NAME = f"{EXP_NAME}-{MODEL_NAME.split('/')[-1]}"
OUTPUT_PATH = f"output/causal/{EXP_NAME}"
MODEL_OUTPUT_PATH = f"{OUTPUT_PATH}/trained_model"
ENV_PATH = "env_file"
LOAD_MODEL_PATH = "output/causal/e001-baseline/trained_model/e001-baseline-gemma-2-9b-it-bnb-4bit/lora_model"


# experiment parameter
DEBUG = False
TRAINING = True
UPLOAD_DATA_TO_S3 = True
UPLOAD_DATA_TO_KAGGLE = True
REMOVE_LOCAL_FILE = False
WANDB = True

# USE_FOLD = 0
# USE_DATA_RATE = 1.0
# VALID_DATA_SIZE = 3000

# model parameter
TRAINING_MAX_LENGTH = 512
SEED = 42
EPOCH = 1
LR = 2e-04
TRAIN_BS = 4  # 16
GRAD_ACC_STEP = 128 // TRAIN_BS  # 仮想的なバッチサイズはTRAIN_BS * GRAD_ACC_STEPとなる
EVAL_BS = 4  # 16
# NUM_LABELS = 3

FREEZE_LAYERS = (
    0  # there're 42 layers in total, we don't add adapters to the first 16 layers
)

# rola parameter
LORA_R = 16  # TODO: 64
LORA_ALPHA = LORA_R * 2
LORA_DROPOUT = 0.05
LORA_BIAS = "none"

RESUME_FROM_CHECKPOINT = False  # 途中から再開する場合はTrueにする

In [4]:
!nvidia-smi

Sat Oct  5 04:25:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-16GB           On  | 00000000:25:00.0 Off |                    0 |
| N/A   18C    P0              24W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
!python --version

Python 3.10.14


In [6]:
def resolve_path(base_path: str) -> str:
    import os

    cwd = os.getcwd()
    print(cwd)
    if cwd == f"/notebooks":
        print("Jupyter Kernel By VSCode!")
        return f"/notebooks/{COMPETITION_NAME}/{base_path}"
    elif cwd == f"/notebooks/{COMPETITION_NAME}":
        print("nohup!")
        return base_path
    elif cwd == f"/notebooks/{COMPETITION_NAME}/{COMPETITION_NAME}/exp":
        print("Jupyter Lab!")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/reranker":
        print("VastAi! Reranker")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/retriever":
        print("VastAi! Retriever")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/causal":
        print("VastAi! Causal")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}":
        print("VastAi!")
        return base_path
    else:
        raise Exception("Unknown environment")


DATA_PATH = resolve_path(DATA_PATH)
print(DATA_PATH)
OUTPUT_PATH = resolve_path(OUTPUT_PATH)
print(OUTPUT_PATH)
MODEL_OUTPUT_PATH = resolve_path(MODEL_OUTPUT_PATH)
print(MODEL_OUTPUT_PATH)
ENV_PATH = resolve_path(ENV_PATH)
print(ENV_PATH)
LOAD_MODEL_PATH = resolve_path(LOAD_MODEL_PATH)
print(LOAD_MODEL_PATH)

/root/eedi-mining-misconceptions-in-mathematics/exp/causal
VastAi! Causal
../../data
/root/eedi-mining-misconceptions-in-mathematics/exp/causal
VastAi! Causal
../../output/causal/e001-baseline
/root/eedi-mining-misconceptions-in-mathematics/exp/causal
VastAi! Causal
../../output/causal/e001-baseline/trained_model
/root/eedi-mining-misconceptions-in-mathematics/exp/causal
VastAi! Causal
../../env_file
/root/eedi-mining-misconceptions-in-mathematics/exp/causal
VastAi! Causal
../../output/causal/e001-baseline/trained_model/e001-baseline-gemma-2-9b-it-bnb-4bit/lora_model


In [7]:
def validate_dataset_name(dataset_name: str) -> None:
    if len(dataset_name) < 6 or len(dataset_name) > 50:
        raise Exception(
            f"データセットの文字列は6~50文字にしてください。現在{len(DATASET_NAME)}文字"
        )
    if "_" in dataset_name:
        raise Exception("datasetの名称に_の使用は禁止です")


validate_dataset_name(DATASET_NAME)

# import

In [8]:
import os
import random

import polars as pl
import numpy as np

import wandb

import torch
from datasets import Dataset
from transformers import TrainingArguments

from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [9]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
NUM_PROC = 16  # os.cpu_count()

In [28]:
import torch
import transformers
import datasets
import trl

assert torch.__version__ == "2.4.1+cu121"
# assert transformers.__version__ == "4.44.2"
assert transformers.__version__ == "4.45.1"
assert datasets.__version__ == "3.0.0"
assert trl.__version__ == "0.11.1"

AssertionError: 

In [11]:
# Seed the same seed to all
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(SEED)

In [12]:
from dotenv import load_dotenv

load_dotenv(f"{ENV_PATH}/.env")

True

# Wandb

In [13]:
if WANDB:
    wandb.login(key=os.environ["WANDB_API_KEY"])
    wandb.init(project=COMPETITION_NAME, name=EXP_NAME)
    REPORT_TO = "wandb"
else:
    REPORT_TO = "none"

REPORT_TO

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: sinchir0. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


'wandb'

# Model Tokenizer Load

In [14]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=TRAINING_MAX_LENGTH,
    dtype=None,
    load_in_4bit=True,
)

==((====))==  Unsloth 2024.9.post3: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla V100-PCIE-16GB. Max memory: 15.773 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [15]:
model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_R,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=LORA_ALPHA,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

Unsloth 2024.9.post3 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


# Data Import & Preprocess

In [16]:
train_long = pl.read_csv(f"{DATA_PATH}/train_long.csv").filter(
    (pl.col("CorrectAnswer") != pl.col("AnswerAlphabet"))
    & (pl.col("MisconceptionId").is_not_null())
)

In [17]:
if DEBUG:
    train_long = train_long.head(100)

In [18]:
train_long

QuestionId,ConstructName,SubjectName,QuestionText,CorrectAnswer,CorrectAnswerText,AnswerType,AnswerText,AllText,AnswerAlphabet,QuestionId_Answer,MisconceptionId,MisconceptionName
i64,str,str,str,str,str,str,str,str,str,str,i64,str
0,"""Use the order of operations to…","""BIDMAS""","""\[ 3 \times 2+4-5 \] Where do …","""A""","""\( 3 \times(2+4)-5 \)""","""AnswerDText""","""Does not need brackets""","""Use the order of operations to…","""D""","""0_D""",1672,"""Confuses the order of operatio…"
1000,"""Simplify an algebraic fraction…","""Simplifying Algebraic Fraction…","""Simplify the following, if pos…","""B""","""\( -1 \)""","""AnswerAText""","""\( t \)""","""Simplify an algebraic fraction…","""A""","""1000_A""",891,"""Incorrectly cancels what they …"
1000,"""Simplify an algebraic fraction…","""Simplifying Algebraic Fraction…","""Simplify the following, if pos…","""B""","""\( -1 \)""","""AnswerCText""","""\( 1 \)""","""Simplify an algebraic fraction…","""C""","""1000_C""",891,"""Incorrectly cancels what they …"
1000,"""Simplify an algebraic fraction…","""Simplifying Algebraic Fraction…","""Simplify the following, if pos…","""B""","""\( -1 \)""","""AnswerDText""","""Does not simplify""","""Simplify an algebraic fraction…","""D""","""1000_D""",353,"""Does not recognise when one pa…"
1001,"""Round numbers to two decimal p…","""Rounding to Decimal Places""","""What is \( \mathbf{3 . 5 1 6 3…","""B""","""\( 3.52 \)""","""AnswerAText""","""\( 3.51 \)""","""Round numbers to two decimal p…","""A""","""1001_A""",1379,"""Rounds down instead of up"""
…,…,…,…,…,…,…,…,…,…,…,…,…
99,"""Given the perimeter, work out …","""Volume and Capacity Units""","""A regular pentagon has a total…","""C""","""\( 3.4 \mathrm{~cm} \)""","""AnswerBText""","""\( 12 \mathrm{~cm} \)""","""Given the perimeter, work out …","""B""","""99_B""",1815,"""When given the perimeter of a …"
99,"""Given the perimeter, work out …","""Volume and Capacity Units""","""A regular pentagon has a total…","""C""","""\( 3.4 \mathrm{~cm} \)""","""AnswerDText""","""Not enough information""","""Given the perimeter, work out …","""D""","""99_D""",255,"""Does not know the meaning of t…"
9,"""Identify horizontal translatio…","""Transformations of functions i…","""What transformation maps the g…","""C""","""Translation by vector \( \left…","""AnswerAText""","""Translation by vector \( \left…","""Identify horizontal translatio…","""A""","""9_A""",1889,"""Believes f(x - a) translates t…"


In [19]:
# TODO: CoT(think step by step)をどうFine-Tuningに反映させるべきかを調べる
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Identify the misconception behind Incorrect Answer.

### Input:
Question: {question_text}
Incorrect Answer: {incorrect_answer_text}
Correct Answer: {correct_answer_text}
Construct Name: {construct_name}
Subject Name: {subject_name}

### Response:
{misconception_name}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    texts = []
    for (
        question_text,
        incorrect_answer_text,
        correct_answer_text,
        construct_name,
        subject_name,
        misconception_name,
    ) in zip(
        examples["QuestionText"],
        examples["AnswerText"],  # incorrect_answer_text
        examples["CorrectAnswerText"],
        examples["ConstructName"],
        examples["SubjectName"],
        examples["MisconceptionName"],
    ):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = (
            alpaca_prompt.format(
                question_text=question_text,
                incorrect_answer_text=incorrect_answer_text,
                correct_answer_text=correct_answer_text,
                construct_name=construct_name,
                subject_name=subject_name,
                misconception_name=misconception_name,
            )
            + EOS_TOKEN
        )
        texts.append(text)
    return {
        "text": texts,
    }


train = Dataset.from_polars(train_long).map(
    formatting_prompts_func,
    batched=True,
)

Map:   0%|          | 0/4370 [00:00<?, ? examples/s]

In [20]:
train, valid = (
    train.filter(lambda example: example["QuestionId"] % 3 != 0, num_proc=NUM_PROC),
    train.filter(lambda example: example["QuestionId"] % 3 == 0, num_proc=NUM_PROC),
)

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Filter (num_proc=16):   0%|          | 0/4370 [00:00<?, ? examples/s]

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Filter (num_proc=16):   0%|          | 0/4370 [00:00<?, ? examples/s]

In [21]:
print(train)
print(valid)

Dataset({
    features: ['QuestionId', 'ConstructName', 'SubjectName', 'QuestionText', 'CorrectAnswer', 'CorrectAnswerText', 'AnswerType', 'AnswerText', 'AllText', 'AnswerAlphabet', 'QuestionId_Answer', 'MisconceptionId', 'MisconceptionName', 'text'],
    num_rows: 2910
})
Dataset({
    features: ['QuestionId', 'ConstructName', 'SubjectName', 'QuestionText', 'CorrectAnswer', 'CorrectAnswerText', 'AnswerType', 'AnswerText', 'AllText', 'AnswerAlphabet', 'QuestionId_Answer', 'MisconceptionId', 'MisconceptionName', 'text'],
    num_rows: 1460
})


In [22]:
print(train[0]["text"])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Identify the misconception behind Incorrect Answer.

### Input:
Question: Simplify the following, if possible: \( \frac{1-t}{t-1} \)
Incorrect Answer: \( t \)
Correct Answer: \( -1 \)
Construct Name: Simplify an algebraic fraction by factorising the numerator
Subject Name: Simplifying Algebraic Fractions

### Response:
Incorrectly cancels what they believe is a factor in algebraic fractions<eos>


In [23]:
print(train[10]["text"])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Identify the misconception behind Incorrect Answer.

### Input:
Question: How many rectangles would you need to shade so that \( 22 \% \) of this shape is shaded? \begin{tabular}{|l|l|l|l|l|l|l|l|l|l|}
\hline & \(\space\) & \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) & \\
\hline & \(\space\) & \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) & \\
\hline & \(\space\) & \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) & \\
\hline & \(\space\) & \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) & \\
\hline & \(\space\) & \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) &  \(\space\) & \\
\hline
\end{

# Model

In [24]:
# training_arguments = TrainingArguments(
#     per_device_train_batch_size=TRAIN_BS,
#     gradient_accumulation_steps=TRAIN_BS,
#     warmup_steps=5,
#     num_train_epochs=EPOCH,
#     learning_rate=LR,
#     fp16=not is_bfloat16_supported(),
#     bf16=is_bfloat16_supported(),
#     logging_steps=1,
#     optim="adamw_8bit",
#     weight_decay=0.01,
#     lr_scheduler_type="linear",
#     seed=SEED,
#     output_dir=MODEL_OUTPUT_PATH,
#     run_name=EXP_NAME,
# )

training_arguments = TrainingArguments(
    output_dir=MODEL_OUTPUT_PATH,
    learning_rate=LR,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    per_device_train_batch_size=TRAIN_BS,
    gradient_accumulation_steps=GRAD_ACC_STEP,
    eval_accumulation_steps=GRAD_ACC_STEP,
    per_device_eval_batch_size=EVAL_BS,
    num_train_epochs=EPOCH,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=0.1,
    save_strategy="steps",
    save_steps=0.1,
    save_total_limit=1,
    logging_steps=2,
    optim="adamw_8bit",
    seed=SEED,
    metric_for_best_model="eval_loss",
    warmup_ratio=0.1,
    lr_scheduler_type="cosine_with_restarts",
    report_to=REPORT_TO,
    run_name=EXP_NAME,
    load_best_model_at_end=True,
    fp16_full_eval=not is_bfloat16_supported(),
    gradient_checkpointing=True,
)

In [25]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train,
    eval_dataset=valid,
    dataset_text_field="text",
    max_seq_length=TRAINING_MAX_LENGTH,
    dataset_num_proc=NUM_PROC,
    packing=False,  # Can make training 5x faster for short sequences.
    args=training_arguments,
)

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Map (num_proc=16):   0%|          | 0/2910 [00:00<?, ? examples/s]

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Map (num_proc=16):   0%|          | 0/1460 [00:00<?, ? examples/s]

In [26]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla V100-PCIE-16GB. Max memory = 15.773 GB.
6.576 GB of memory reserved.


In [27]:
if TRAINING:
    trainer_stats = trainer.train()
else:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=LOAD_MODEL_PATH,
        max_seq_length=TRAINING_MAX_LENGTH,
        dtype=None,  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
        load_in_4bit=True,  # Use 4bit quantization to reduce memory usage. Can be False.
    )

    FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,910 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 32
\        /    Total batch size = 128 | Total steps = 22
 "-____-"     Number of trainable parameters = 54,018,048


Step,Training Loss,Validation Loss
3,2.908900,1.954767


E1005 04:30:32.054000 139763982849856 torch/_subclasses/fake_tensor.py:1761] [1/4] failed while attempting to run meta for aten.add_.Tensor
E1005 04:30:32.054000 139763982849856 torch/_subclasses/fake_tensor.py:1761] [1/4] Traceback (most recent call last):
E1005 04:30:32.054000 139763982849856 torch/_subclasses/fake_tensor.py:1761] [1/4]   File "/opt/conda/lib/python3.10/site-packages/torch/_subclasses/fake_tensor.py", line 1757, in _dispatch_impl
E1005 04:30:32.054000 139763982849856 torch/_subclasses/fake_tensor.py:1761] [1/4]     r = func(*args, **kwargs)
E1005 04:30:32.054000 139763982849856 torch/_subclasses/fake_tensor.py:1761] [1/4]   File "/opt/conda/lib/python3.10/site-packages/torch/_ops.py", line 667, in __call__
E1005 04:30:32.054000 139763982849856 torch/_subclasses/fake_tensor.py:1761] [1/4]     return self_._op(*args, **kwargs)
E1005 04:30:32.054000 139763982849856 torch/_subclasses/fake_tensor.py:1761] [1/4]   File "/opt/conda/lib/python3.10/site-packages/torch/_meta_r

TorchRuntimeError: Failed running call_function <built-in function iadd>(*(FakeTensor(..., device='cuda:0', size=(s10, s2*s3, s17, s11),
           dtype=torch.float16), FakeTensor(..., device='cuda:0', size=(s18, 1, s19, s20), dtype=torch.float16)), **{}):
Attempting to broadcast a dimension of length s20 at -1! Mismatching argument at index 1 had torch.Size([s18, 1, s19, s20]); but expected shape should be broadcastable to [s10, s2*s3, s17, s11]

from user code:
   File "/opt/conda/lib/python3.10/site-packages/unsloth/kernels/flex_attention.py", line 66, in slow_attention_softcapping
    A += causal_mask[:q_len, :q_len]

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


# Save

In [ ]:
model.save_pretrained(f"{MODEL_OUTPUT_PATH}/lora_model")  # Local saving
tokenizer.save_pretrained(f"{MODEL_OUTPUT_PATH}/lora_model")

# AWSへのアップロード

In [ ]:
# S3へのアップロード
if not DEBUG and UPLOAD_DATA_TO_S3:
    # uninstall
    !sudo rm /usr/bin/aws
    !sudo rm /usr/bin/aws_completer
    !sudo rm -rf /usr/local/aws-cli

    # install
    !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
    !unzip -o -qq awscliv2.zip
    !sudo ./aws/install --update

    # upload
    output_name = MODEL_OUTPUT_PATH.split("/")[-1]
    os.system(
        f"aws s3 cp --recursive {MODEL_OUTPUT_PATH} s3://{COMPETITION_NAME}/trained_model/{output_name}"
    )

In [43]:
# ダウンロード（参考）
# !sudo rm /usr/bin/aws
# !sudo rm /usr/bin/aws_completer
# !sudo rm -rf /usr/local/aws-cli

# !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
# !unzip -o -qq awscliv2.zip
# !sudo ./aws/install --update

# !aws s3 cp --recursive s3://automated-essay-scoring/trained_model/e005-regression /notebooks/automated_essay_scoring/trained_models/e005-regression

# Kaggle Datasetへのupload

In [ ]:
import os

os.system("mkdir -p ~/.kaggle/")
os.system(f"cp /{ENV_PATH}/kaggle.json ~/.kaggle/")
os.system("chmod 600 ~/.kaggle/kaggle.json")

In [ ]:
if not DEBUG and UPLOAD_DATA_TO_KAGGLE:
    import os
    import json

    from kaggle.api.kaggle_api_extended import KaggleApi

    def dataset_create_new(dataset_name: str, upload_dir: str):
        # if "_" in dataset_name:
        #     raise ValueError("datasetの名称に_の使用は禁止です")
        dataset_metadata = {}
        dataset_metadata["id"] = f"sinchir0/{dataset_name}"
        dataset_metadata["licenses"] = [{"name": "CC0-1.0"}]
        dataset_metadata["title"] = dataset_name
        with open(os.path.join(upload_dir, "dataset-metadata.json"), "w") as f:
            json.dump(dataset_metadata, f, indent=4)
        api = KaggleApi()
        api.authenticate()
        api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode="tar")

    print(f"Create Dataset name:{DATASET_NAME}, output_dir:{MODEL_OUTPUT_PATH}")
    dataset_create_new(dataset_name=DATASET_NAME, upload_dir=MODEL_OUTPUT_PATH)

# ローカルからのデータの削除

In [39]:
# if not DEBUG and REMOVE_LOCAL_FILE:
#     # ローカルからは削除
#     os.system(f"rm -rf {MODEL_OUTPUT_PATH}")

In [40]:
if WANDB:
    wandb.finish()

In [41]:
print("finish Notebook!")